# Prepair french lpis data for multi-year crop classification

preprocess goal is to obtain stable land parcel from multi year farmer lpis declaration and also filter some corp parcel.

* filter parcel by size /elongation
* keep up unique id for future analysis
* convert crop code to new nomenclature (73 classes)
* find and export "stable" parcel.

In [1]:
import fnmatch
import os
import glob

import pandas as pd
import geopandas as gpd

### set up data dir

In [2]:
lpis_shp_dir = "/media/ndavid/54F6F347F6F3283E1/data/EXP_ASP/rotation/SHP"
years = [2018,2019,2020]
admin_id = [1, 71] # french departement code
nomenclature_root_dir = "/media/ndavid/54F6F347F6F3283E1/data/EXP_ASP/rotation/NOMENCLATURE"

### data file name from year and admin limit

In [10]:
def lpis_filename_fr(root_dir, year, dep_id) :
    """
    define standard french lpis filename from year and administrative limite id
    
    root_dir (str) : path to lpis shapefile
    year (int ) : year of parcel monitoring
    dep_id (int) : french administrative limite id (french departement)
    
    """
    template_prefix = f"SURFACES-{year}-PARCELLES-GRAPHIQUES-CONSTATEES_{dep_id:03}"
    template_path = os.path.join(root_dir, template_prefix)
    shp_filename = glob.glob(f'{template_path}*.shp')
    if shp_filename :
        return shp_filename[0]
    else:
        return None

for dep in admin_id :
    print(f"departement {dep:03} :")
    for year in years : 
        lpis_file = lpis_filename_fr(lpis_shp_dir, year, dep)
        print(f"\t  {year}: {lpis_file}")


departement 001 :
	  2018: /media/ndavid/54F6F347F6F3283E1/data/EXP_ASP/rotation/SHP/SURFACES-2018-PARCELLES-GRAPHIQUES-CONSTATEES_001_20210301.shp
	  2019: /media/ndavid/54F6F347F6F3283E1/data/EXP_ASP/rotation/SHP/SURFACES-2019-PARCELLES-GRAPHIQUES-CONSTATEES_001_20210228.shp
	  2020: /media/ndavid/54F6F347F6F3283E1/data/EXP_ASP/rotation/SHP/SURFACES-2020-PARCELLES-GRAPHIQUES-CONSTATEES_001_20210227.shp
departement 071 :
	  2018: /media/ndavid/54F6F347F6F3283E1/data/EXP_ASP/rotation/SHP/SURFACES-2018-PARCELLES-GRAPHIQUES-CONSTATEES_071_20210308.shp
	  2019: /media/ndavid/54F6F347F6F3283E1/data/EXP_ASP/rotation/SHP/SURFACES-2019-PARCELLES-GRAPHIQUES-CONSTATEES_071_20210308.shp
	  2020: /media/ndavid/54F6F347F6F3283E1/data/EXP_ASP/rotation/SHP/SURFACES-2020-PARCELLES-GRAPHIQUES-CONSTATEES_071_20210308.shp


### read, transform ini lpis file to target lpis file

##### read lpis data

First we load ini lpis file into a dataframe, then we skip useless columns and create an unique id columns.

In [8]:
df_lpis_a = gpd.read_file(lpis_filename_fr(lpis_shp_dir, 2020, 1)) 

In [11]:
def format_lpis_french(geodf) :
    # skip unused columns 
    all_remove_columns = [
       'PRECISION', 
       'RECONVER_P',
       'RETOURNMT_',
       'SEMENCE',
       'DEST_ICHN',
       'CULTURE_D1',
       'CULTURE_D2',
       'BIO',
       'MARAICHAGE',
       'AGROFOREST',
       'FORCE_MAJE',
       'ENGAGEMENT'
    ]
    remove_columns = [column for column in all_remove_columns if column in geodf.columns]
    
    geodf = geodf.drop(columns=remove_columns)

    # set unique id from pacage (farm id) num_ilot and num_parcel 
    # "ilot" are a group of multiple parcels. Ilot boundary should not change (road, building etc..)
    # but parcel boundary inside ilot could change/move from year to year.
    geodf['UID'] = geodf.agg(lambda x: f"{x['PACAGE']}{x['NUM_ILOT']:04}{x['NUM_PARCEL']:03}", axis=1)
    geodf = geodf.drop(columns=['PACAGE', 'NUM_ILOT', 'NUM_PARCEL'])

    return geodf                          

In [12]:
df_lpis_a = format_lpis_french(df_lpis_a)
df_lpis_a.head()

,CODE_CULTU,SURF_ADM,geometry,UID
0,PPH,0.56,"POLYGON ((912584.420 6526025.300, 912550.160 6...",0010178900071001
1,PPH,0.42,"POLYGON ((912386.090 6526281.830, 912354.610 6...",0010178900072001
2,PPH,0.32,"POLYGON ((912504.430 6526382.900, 912370.090 6...",0010178900073001
3,PPH,0.59,"POLYGON ((911918.510 6527850.830, 911787.380 6...",0010178900086001
4,PPH,1.99,"POLYGON ((915304.960 6524237.410, 915278.040 6...",0010178900002001


##### transform lpis data

Then we need :
* to convert the initial crop code (more than 300 codes) to a target crop code. This is done with a lookup table    between ini crop code and target crop code
* to add additional columns for **ini** code for easiest display/label on GIS or plot (crop label translated to english, label with fixed length)
* to add additional columns for **target** code for easiest display/label on GIS or plot (crop label translated to english, label with fixed length)

For this we use 3 additionals file (each is a csv file) :
  
   1. a lookup table between ini and target crop code.
   2. a nomenclature file with additional columns for target crop code.
   3. a nomenclature file with additional columns for ini crop code.


In [13]:
# load both french ini crop nomenclature and lookup table to convert crop code    
def load_lpis_code_fr(root_dir, target_nomenclature="N73"):
    lpis_code_fr_df = pd.read_csv(
        os.path.join(root_dir, "FR", "code_cultu_2019.csv"),
        skiprows=9, sep=";")
    lookup_table_fr_n73 = pd.read_csv(
        os.path.join(root_dir, "FR", "cultu2n73.csv"),
        skiprows=10, sep=";")
    return lpis_code_fr_df, lookup_table_fr_n73

lpis_code_fr_df, lookup_table_fr_n73_df = load_lpis_code_fr(nomenclature_root_dir, target_nomenclature="N73")

In [14]:
print("French lpis crop code ")
lpis_code_fr_df.head(5)

French lpis crop code 


,Code_cultu,Libelle,LABEL_EN,Code_EN
0,ACA,Autre culture non précisée (DOM),other cultivation,OCU
1,AGR,Agrume,Citrus,CIT
2,AIL,Ail,Garlic,GAR
3,ANA,Ananas,pineapple,PIN
4,ANE,Aneth,dill,DIL


In [15]:
print("lookup table")
lookup_table_fr_n73_df.head()  

lookup table


,Code_cultu,Libelle,Label_2019_V1,CODE9
0,ACA,Autre culture non précisée (DOM),Autres,AUTRES
1,BAR,Bardane [Asteracée bisannuelle],Autres,AUTRES
2,CID,Cultures conduites en inter-rang,Autres,AUTRES
3,CIT,Cultures conduites en inter-rang,Autres,AUTRES
4,CRN,Cornille [Fabacée - plante herbacée annuelle],Autres,AUTRES


In [16]:
# load target crop nomenclature
n73_code_df = pd.read_csv(os.path.join(nomenclature_root_dir, "nomenclature_73c_2019.csv"), skiprows=12, sep=";")
print("Target crop nomenclature ")
n73_code_df.head(5)

Target crop nomenclature 


,LABEL_2019_V1,CODE9,CODE3,N2019,LABEL_2019_ENG,CODE9_EN
0,Herbe prédominante,HERB_PRED,HER,1,Grass prevailing,GRASS_PRE
1,Blé tendre d’hiver,BLE_TEN_H,BTH,2,Winter bread wheat,WHEAT_B_W
2,Maïs,MAIS,MIS,3,Maize,MAIZE
3,Orge d'hiver,ORGE_H,ORH,4,Winter barley,BARLEY_W
4,Colza d’hiver,COLZA_H,CZH,5,Winter rapeseed,RAPE_W


Once all files/data are load on dataframe we JOIN/merge all columns to lpis data.

In [17]:
def convert_code_lpis_french(
    geodf, nomenclature_df, lookup_df, lpis_code_df= None, code_ini="CODE_CULTU",
    lookup_ini="Code_cultu", lookup_target="CODE9", nomenclature_ini="CODE9", lpis_code_df_key="Code_cultu") :
    
    # join lpis df and lookup dataframe with lookup key columns
    lookup_df = lookup_df.drop(columns=["Label_2019_V1", "Libelle"])
    geodf = geodf.merge(lookup_df, how="left", left_on=code_ini, right_on=lookup_ini)
    if lookup_ini != code_ini:
        geodf = geodf.drop(columns=[lookup_ini])
    
    # join with target nomenclature df
    geodf = geodf.merge(nomenclature_df, how="left", left_on=lookup_target, right_on=nomenclature_ini)
    if lookup_target != nomenclature_ini:
        geodf = geodf.drop(columns=[lookup_target])
        
    if lpis_code_df is not None:
        # add english translation for original french code and label
        geodf = geodf.merge(lpis_code_df, how="left", left_on=code_ini, right_on=lpis_code_df_key)
        if lpis_code_df_key != code_ini:
            geodf = geodf.drop(columns=[lpis_code_df_key])
        
    # rename and force code number to int
    geodf = geodf.rename(columns={
        "LABEL_2019_V1": "LABEL_FR", 
        "Libelle": "LABEL_INI_FR",
        "CODE_CULTU": "CODE_INI_FR",
        "Code_EN": "CODE_INI_EN",
        "N2019": "CODE_NUM",
        "CODE9": "CODE9_FR",
        "LABEL_2019_ENG": "LABEL",
        "LABEL_EN": "LABEL_INI_EN"

    })
    geodf = geodf.drop(columns=["CODE3"])
    geodf = geodf.rename(columns={"CODE9_EN": "CODE9"})
        
    cols = list(geodf.columns.values)
    geodf = geodf[[
        'UID','CODE_NUM','CODE9','LABEL',
        'CODE9_FR','LABEL_FR',
        'CODE_INI_FR','LABEL_INI_FR','LABEL_INI_EN','CODE_INI_EN',
        'SURF_ADM', 'geometry']]
    
    geodf["CODE_NUM"] = geodf["CODE_NUM"].astype('int')

    return geodf


In [19]:
df_lpis_b = convert_code_lpis_french(
    df_lpis_a, n73_code_df, lookup_df=lookup_table_fr_n73_df, lpis_code_df=lpis_code_fr_df)
df_lpis_b.head()

,UID,CODE_NUM,CODE9,LABEL,CODE9_FR,LABEL_FR,CODE_INI_FR,LABEL_INI_FR,LABEL_INI_EN,CODE_INI_EN,SURF_ADM,geometry
0,0010178900071001,1,GRASS_PRE,Grass prevailing,HERB_PRED,Herbe prédominante,PPH,Prairie permanente - herbe prédominante,permanent grassland,PGR,0.56,"POLYGON ((912584.420 6526025.300, 912550.160 6..."
1,0010178900072001,1,GRASS_PRE,Grass prevailing,HERB_PRED,Herbe prédominante,PPH,Prairie permanente - herbe prédominante,permanent grassland,PGR,0.42,"POLYGON ((912386.090 6526281.830, 912354.610 6..."
2,0010178900073001,1,GRASS_PRE,Grass prevailing,HERB_PRED,Herbe prédominante,PPH,Prairie permanente - herbe prédominante,permanent grassland,PGR,0.32,"POLYGON ((912504.430 6526382.900, 912370.090 6..."
3,0010178900086001,1,GRASS_PRE,Grass prevailing,HERB_PRED,Herbe prédominante,PPH,Prairie permanente - herbe prédominante,permanent grassland,PGR,0.59,"POLYGON ((911918.510 6527850.830, 911787.380 6..."
4,0010178900002001,1,GRASS_PRE,Grass prevailing,HERB_PRED,Herbe prédominante,PPH,Prairie permanente - herbe prédominante,permanent grassland,PGR,1.99,"POLYGON ((915304.960 6524237.410, 915278.040 6..."


check that all parcel have a corresponding code on crop code.

In [20]:
df_lpis_b[df_lpis_b['CODE_NUM'].isna()]

,UID,CODE_NUM,CODE9,LABEL,CODE9_FR,LABEL_FR,CODE_INI_FR,LABEL_INI_FR,LABEL_INI_EN,CODE_INI_EN,SURF_ADM,geometry


### filter lpis data
we need to filter data with three steps :
 1. Filter parcel too small or too elongated.
 2. Filter parcel belonging to target crop code where there is not enough data.
 3. Filter stable during three years.
 
73 classes <- au moins 100 parcelles sur chaque année
liste des parcelles stables (les limites ont pas bougés de l'ordre du pixels)
 

##### filter small/elongated parcel
filter parcel with :
 * area_threshold = 800 m²
 * area_over_perim_threshold = 10m
 * area_over_bbox_threshold = .2

In [21]:
def filter_small_parcel(
    lpis_df, area_threshold=800, area_over_perim_threshold=10, area_over_bbox_threshold=0.2 ) :
    
    lpis_df["area"] = lpis_df['geometry'].area
    lpis_df["elon_thr"] = lpis_df['geometry'].area/lpis_df['geometry'].length
    lpis_df["bbox_thr"] = lpis_df['geometry'].area/lpis_df['geometry'].envelope.area
    lpis_df = lpis_df.loc[
         (lpis_df.area > area_threshold) & 
         (lpis_df.elon_thr > area_over_perim_threshold) & 
         (lpis_df.bbox_thr > area_over_bbox_threshold) ]
    return lpis_df

In [22]:
df_lpis_filter = filter_small_parcel(df_lpis_b)
df_lpis_filter.head(10)

,UID,CODE_NUM,CODE9,LABEL,CODE9_FR,LABEL_FR,CODE_INI_FR,LABEL_INI_FR,LABEL_INI_EN,CODE_INI_EN,SURF_ADM,geometry,area,elon_thr,bbox_thr
0,0010178900071001,1,GRASS_PRE,Grass prevailing,HERB_PRED,Herbe prédominante,PPH,Prairie permanente - herbe prédominante,permanent grassland,PGR,0.56,"POLYGON ((912584.420 6526025.300, 912550.160 6...",5636.461350,16.924689,0.493283
1,0010178900072001,1,GRASS_PRE,Grass prevailing,HERB_PRED,Herbe prédominante,PPH,Prairie permanente - herbe prédominante,permanent grassland,PGR,0.42,"POLYGON ((912386.090 6526281.830, 912354.610 6...",4384.349150,15.282616,0.490829
3,0010178900086001,1,GRASS_PRE,Grass prevailing,HERB_PRED,Herbe prédominante,PPH,Prairie permanente - herbe prédominante,permanent grassland,PGR,0.59,"POLYGON ((911918.510 6527850.830, 911787.380 6...",5940.632650,16.765193,0.600527
4,0010178900002001,1,GRASS_PRE,Grass prevailing,HERB_PRED,Herbe prédominante,PPH,Prairie permanente - herbe prédominante,permanent grassland,PGR,1.99,"POLYGON ((915304.960 6524237.410, 915278.040 6...",24842.491400,37.643952,0.607086
5,0010178900009001,1,GRASS_PRE,Grass prevailing,HERB_PRED,Herbe prédominante,J5M,Jachère de 5 ans ou moins,fallow less 5 year,FW5,2.37,"POLYGON ((914928.759 6524561.129, 914921.062 6...",23842.630177,31.438654,0.571198
7,0010178900023002,19,SOYBEAN,Soybean,SOJA,Soja,SOJ,Soja,soybean,SOY,5.59,"POLYGON ((914518.010 6525923.050, 914452.560 6...",54634.508201,56.758116,0.791493
8,0010178900027001,3,MAIZE,Maize,MAIS,Maïs,MIS,Maïs,Maize,MIZ,0.84,"POLYGON ((915300.010 6525906.570, 915258.980 6...",8413.730700,16.034399,0.455357
9,0010178900041001,3,MAIZE,Maize,MAIS,Maïs,MIS,Maïs,Maize,MIZ,0.60,"POLYGON ((915402.590 6525805.190, 915401.500 6...",5966.363100,16.852660,0.460003
10,0010178900043002,1,GRASS_PRE,Grass prevailing,HERB_PRED,Herbe prédominante,PPH,Prairie permanente - herbe prédominante,permanent grassland,PGR,0.57,"POLYGON ((914302.360 6524919.130, 914275.490 6...",5712.627750,15.787777,0.355599
11,0010178900043003,1,GRASS_PRE,Grass prevailing,HERB_PRED,Herbe prédominante,PPH,Prairie permanente - herbe prédominante,permanent grassland,PGR,0.72,"POLYGON ((914393.830 6525092.930, 914373.800 6...",7188.967650,14.080782,0.356492


load all data on experimentation area to compute crop stat by year

In [23]:
lpis_df_dict = {}
for year in years :
    lpis_df_year_list = []
    # print(year)
    for dep_id in admin_id :
        # print(dep_id)
        lpis_df_year_dep = gpd.read_file(lpis_filename_fr(lpis_shp_dir, year, dep_id))
        lpis_df_year_dep = format_lpis_french(lpis_df_year_dep)
        lpis_df_year_dep_transform = convert_code_lpis_french(
            lpis_df_year_dep, n73_code_df, lookup_df=lookup_table_fr_n73_df, lpis_code_df=lpis_code_fr_df)
        lpis_df_year_dep_filter_small = filter_small_parcel(lpis_df_year_dep_transform)
        lpis_df_year_list.append(lpis_df_year_dep_filter_small)
    
    lpis_df_year = pd.concat(lpis_df_year_list)
    lpis_df_dict[year] = lpis_df_year

##### filter small crop group
 * get stat by year/crop
 * filter crop group by size
 * filter by crop group id
 
 first compute crop statistic for all year. Then concatenate all stat in one dataframe with parcel count by year as dataframe columns.

In [41]:
crop_stat_df_dict = {}
crop_index = n73_code_df["CODE9"]
for year in years :
    tmp_df = lpis_df_dict[year][["CODE_NUM","CODE9_FR"]]
    stat_df_year = tmp_df.groupby(by="CODE9_FR").agg(['count'])
    stat_df_year = stat_df_year.reindex(crop_index, fill_value=0)
    # print(stat_df_year.columns)
    stat_df_year.columns = [f"{year}_count"]
    crop_stat_df_dict[year] = stat_df_year
    # print(stat_df_year.head(5))

crop_stat_df = pd.concat(crop_stat_df_dict.values(), axis=1)

In [57]:
print(crop_stat_df.head(15))

           2018_count  2019_count  2020_count
CODE9                                        
HERB_PRED      116173      115781      116093
BLE_TEN_H       16654       16120       13948
MAIS            22566       22955       22687
ORGE_H           5945        5880        5603
COLZA_H          4816        3275        3739
ORGE_P            126         289         561
LIGN_PRED          31          40          38
TOURNESOL         725        1073        1592
VIGNE            6510        6652        7594
BETTERAVE          28          27          26
LUZERNE          1388        1611        1893
TRITICA_H        4388        4363        4184
BOIS_PATU         250         275         288
BLE_DUR_H         206         107         118
FR_LE_FLE         710         725         773


filter statistic dataframe to keep only crop code with enought parcel for each year (100)

In [53]:
nb_parcel_thr = 100
keeped_crop_code_df = crop_stat_df.loc[
         (crop_stat_df["2018_count"] > nb_parcel_thr) & 
         (crop_stat_df["2019_count"] > nb_parcel_thr) & 
         (crop_stat_df["2020_count"] > nb_parcel_thr) ].sort_values(by="2020_count", ascending=False)

In [55]:
print(keeped_crop_code_df.head(40))

           2018_count  2019_count  2020_count
CODE9                                        
HERB_PRED      116173      115781      116093
MAIS            22566       22955       22687
BLE_TEN_H       16654       16120       13948
VIGNE            6510        6652        7594
ORGE_H           5945        5880        5603
TRITICA_H        4388        4363        4184
COLZA_H          4816        3275        3739
SOJA             3578        3307        3568
LUZERNE          1388        1611        1893
TOURNESOL         725        1073        1592
SURF_TNEX        1452        1343        1390
ML_CEREAL         773         898         917
SORGHO            302         630         889
FR_LE_FLE         710         725         773
LEGU_FOUR         848         797         764
ORGE_P            126         289         561
AVOINE_H          530         413         384
SEIGLE_H          331         319         344
AVOINE_P          152         289         295
BOIS_PATU         250         275 

In [56]:
keeped_crop_codes = keeped_crop_code.index.values
print(keeped_crop_codes)

['HERB_PRED' 'MAIS' 'BLE_TEN_H' 'VIGNE' 'ORGE_H' 'TRITICA_H' 'COLZA_H'
 'SOJA' 'LUZERNE' 'TOURNESOL' 'SURF_TNEX' 'ML_CEREAL' 'SORGHO' 'FR_LE_FLE'
 'LEGU_FOUR' 'ORGE_P' 'AVOINE_H' 'SEIGLE_H' 'AVOINE_P' 'BOIS_PATU'
 'POMM_TERR' 'EPEAUTRE' 'BLE_DUR_H']


In [58]:
lpis_df_filter_dict = {}
for year in years :
    tmp_df = lpis_df_dict[year]
    lpis_df_filter_year = tmp_df[tmp_df['CODE9_FR'].isin(keeped_crop_codes)]
    lpis_df_filter_dict[year]=lpis_df_filter_year

In [62]:
lpis_df_filter_dict[2020].head(5)

,UID,CODE_NUM,CODE9,LABEL,CODE9_FR,LABEL_FR,CODE_INI_FR,LABEL_INI_FR,LABEL_INI_EN,CODE_INI_EN,SURF_ADM,geometry,area,elon_thr,bbox_thr
0,0010178900071001,1,GRASS_PRE,Grass prevailing,HERB_PRED,Herbe prédominante,PPH,Prairie permanente - herbe prédominante,permanent grassland,PGR,0.56,"POLYGON ((912584.420 6526025.300, 912550.160 6...",5636.461350,16.924689,0.493283
1,0010178900072001,1,GRASS_PRE,Grass prevailing,HERB_PRED,Herbe prédominante,PPH,Prairie permanente - herbe prédominante,permanent grassland,PGR,0.42,"POLYGON ((912386.090 6526281.830, 912354.610 6...",4384.349150,15.282616,0.490829
3,0010178900086001,1,GRASS_PRE,Grass prevailing,HERB_PRED,Herbe prédominante,PPH,Prairie permanente - herbe prédominante,permanent grassland,PGR,0.59,"POLYGON ((911918.510 6527850.830, 911787.380 6...",5940.632650,16.765193,0.600527
4,0010178900002001,1,GRASS_PRE,Grass prevailing,HERB_PRED,Herbe prédominante,PPH,Prairie permanente - herbe prédominante,permanent grassland,PGR,1.99,"POLYGON ((915304.960 6524237.410, 915278.040 6...",24842.491400,37.643952,0.607086
5,0010178900009001,1,GRASS_PRE,Grass prevailing,HERB_PRED,Herbe prédominante,J5M,Jachère de 5 ans ou moins,fallow less 5 year,FW5,2.37,"POLYGON ((914928.759 6524561.129, 914921.062 6...",23842.630177,31.438654,0.571198


In [64]:
# export to geosjon to check preprocess so far
lpis_geojson_dir = "/media/ndavid/54F6F347F6F3283E1/data/EXP_ASP/rotation/GEOJSON"
for year in years :
    lpis_df_filter_dict[year].to_file(os.path.join(lpis_geojson_dir, f"lpis_{year}_filter.geojson"), driver='GeoJSON')


##### filter stable parcel for three year


In [81]:
end_year_columns = ["UID", "CODE_NUM", "CODE9", "CODE_INI_FR"]
lpis_2020_df = lpis_df_filter_dict[2020][end_year_columns + ["geometry"]]
lpis_stable_df = lpis_2020_df.copy()
lpis_stable_df.columns = [f"{column}_2020" for column in end_year_columns] + ["geometry"]

for year in [2019, 2018] :
    filter_df = lpis_df_filter_dict[year][end_year_columns + ["geometry"] ]
    join_df = filter_df.copy()
    join_df.columns = [f"{column}_{year}" for column in end_year_columns] + ["geometry"]
    join_df["geometry"] = filter_df["geometry"].buffer(3)
    lpis_stable_df = gpd.sjoin(lpis_stable_df, join_df, how="inner", op='within')
    lpis_stable_df.rename(columns={"index_right": f"index_{year}"}, inplace=True)

reorder_columns = []
for column in ["CODE9", "CODE_INI_FR", "CODE_NUM", "UID"] :
    for year in years :
        reorder_columns.append(f"{column}_{year}")

lpis_stable_df = lpis_stable_df[reorder_columns + ["geometry"]]
lpis_stable_df.head(10)

,CODE9_2018,CODE9_2019,CODE9_2020,CODE_INI_FR_2018,CODE_INI_FR_2019,CODE_INI_FR_2020,CODE_NUM_2018,CODE_NUM_2019,CODE_NUM_2020,UID_2018,UID_2019,UID_2020,geometry
0,GRASS_PRE,GRASS_PRE,GRASS_PRE,PPH,PPH,PPH,1,1,1,0010178900071001,0010178900071001,0010178900071001,"POLYGON ((912584.420 6526025.300, 912550.160 6..."
1,GRASS_PRE,GRASS_PRE,GRASS_PRE,PPH,PPH,PPH,1,1,1,0010178900072001,0010178900072001,0010178900072001,"POLYGON ((912386.090 6526281.830, 912354.610 6..."
3,GRASS_PRE,GRASS_PRE,GRASS_PRE,PPH,PPH,PPH,1,1,1,0010178900086001,0010178900086001,0010178900086001,"POLYGON ((911918.510 6527850.830, 911787.380 6..."
4,GRASS_PRE,GRASS_PRE,GRASS_PRE,PPH,PPH,PPH,1,1,1,0010178900002001,0010178900002001,0010178900002001,"POLYGON ((915304.960 6524237.410, 915278.040 6..."
5,TRITICA_W,TRITICA_W,GRASS_PRE,TTH,TTH,J5M,12,12,1,0010178900009001,0010178900009001,0010178900009001,"POLYGON ((914928.759 6524561.129, 914921.062 6..."
7,SOYBEAN,MAIZE,SOYBEAN,SOJ,MIS,SOJ,19,3,19,0010178900023002,0010178900023002,0010178900023002,"POLYGON ((914518.010 6525923.050, 914452.560 6..."
8,SOYBEAN,MAIZE,MAIZE,SOJ,MIS,MIS,19,3,3,0010178900027001,0010178900027001,0010178900027001,"POLYGON ((915300.010 6525906.570, 915258.980 6..."
9,SOYBEAN,MAIZE,MAIZE,SOJ,MIS,MIS,19,3,3,0010178900041003,0010178900041001,0010178900041001,"POLYGON ((915402.590 6525805.190, 915401.500 6..."
10,GRASS_PRE,GRASS_PRE,GRASS_PRE,PPH,PPH,PPH,1,1,1,0010178900043002,0010178900043002,0010178900043002,"POLYGON ((914302.360 6524919.130, 914275.490 6..."
11,GRASS_PRE,GRASS_PRE,GRASS_PRE,PPH,PPH,PPH,1,1,1,0010178900043001,0010178900043003,0010178900043003,"POLYGON ((914393.830 6525092.930, 914373.800 6..."


In [82]:
lpis_stable_df.to_file(os.path.join(lpis_geojson_dir, f"lpis_stable_all_years.geojson"), driver='GeoJSON')

In [84]:
# export to gpkg to check preprocess so far
lpis_gpkg_dir = "/media/ndavid/54F6F347F6F3283E1/data/EXP_ASP/rotation/"
for year in years :
    lpis_df_filter_dict[year].to_file(os.path.join(lpis_gpkg_dir, "lpis_31TFM.gpkg"), layer=f'filter_{year}', driver='GPKG')

lpis_stable_df.to_file(os.path.join(lpis_gpkg_dir, "lpis_31TFM.gpkg"), layer='stable_all_years', driver="GPKG")
